In [1]:
# Import dependencies
from skimage import io
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
import numpy as np

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


In [3]:
# Load data
train_volume = io.imread('Data/train-volume.tif')
train_labels = io.imread('Data/train-labels.tif')
test_volume = io.imread('Data/test-volume.tif')

In [4]:
# Normalize function
def normalize(x):
    x = x.astype(np.float32)
    x /=255
    return x

In [5]:
# Normalize data
train_volume = normalize(train_volume)
train_labels = normalize(train_labels)
test_volume = normalize(test_volume)

In [6]:
# Batch function

# The data is currently a multidimensional numpy array (ndarray) with 3 dimensions. It has the shape 30 * 512 * 512, which
# means that the data exists of 30 greyscale pictures with dimension 512 pixels on 512 pixels. We want to make 30 batches of 1 
# picture each. We'll also have to reshape the data to pass it to convolutional layers in tensorflow (4 dimensions required).

def get_batches(data):
    data = np.reshape(data, (30, 512, 512, 1))
    data = np.split(data, 30)
    return data

In [7]:
# Batch data
volume_batches = get_batches(train_volume)
labels_batches = get_batches(train_labels)
test_batches = get_batches(test_volume)

In [ ]:
# Model

inputs = tf.placeholder(dtype = tf.float32, shape=(None, 512, 512, 1), name='inputs')
targets = tf.placeholder(dtype = tf.float32, shape=(None, 512, 512, 1), name='targets')
learning_rate = 0.001
dropout = 0.5

# Now 512x512x1
conv1 = tf.layers.conv2d(inputs, 32, 4, 4, padding='same', activation=tf.tanh)
maxpool1 = tf.layers.max_pooling2d(conv1, 2, 2, padding='same')
dropout1 = tf.layers.dropout(maxpool1, rate=dropout)
# Now 256x256x32
conv2 = tf.layers.conv2d(dropout1, 64, 5, 5, padding='same', activation=tf.tanh)
maxpool2 = tf.layers.max_pooling2d(conv2, 2, 2, padding='same')
dropout2 = tf.layers.dropout(maxpool2, rate=dropout)
# Now 128x128x64
conv3 = tf.layers.conv2d(dropout2, 128, 4, 4, padding='same', activation=tf.tanh)
maxpool3 = tf.layers.max_pooling2d(conv3, 2, 2, padding='same')
dropout3 = tf.layers.dropout(maxpool3, rate=dropout)
# Now 64x64x128
conv4 = tf.layers.conv2d(dropout3, 256, 4, 4, padding='same', activation=tf.tanh)
maxpool4 = tf.layers.max_pooling2d(conv4, 2, 2, padding='same')
dropout4 = tf.layers.dropout(maxpool4, rate=dropout)
# Now 32x32x256
conv5 = tf.layers.conv2d(dropout4, 512, 3, 3, padding='same', activation=tf.tanh)
maxpool5 = tf.layers.max_pooling2d(conv5, 2, 2, padding='same')
dropout5 = tf.layers.dropout(maxpool5, rate=dropout)
# Now 16x16x512

# Flatten it
flat = tf.reshape(dropout5, (-1, 16*16*512))
        
# Logits
logits = tf.layers.dense(flat, 512*512*1)
logits = tf.reshape(logits, (512, 512, 1))

# Output
output = tf.nn.softmax(logits)

# Loss and Optimizer
loss = tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=logits)
cost = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [ ]:
def train_neural_network(session, optimizer, feature_batch, label_batch):
    session.run(optimizer, feed_dict = {inputs: feature_batch, targets: label_batch})

# Function for showing the stats
def print_stats(session, feature_batch, label_batch, cost):
    print("cost: ", session.run(cost, feed_dict = {inputs: feature_batch, targets: label_batch}))

# Hyperparameters
epochs = 10
batch_size = 1
n_batches = len(volume_batches) / batch_size

### Train Model

# Create the session
sess = tf.Session()

# Initializing the variables
sess.run(tf.global_variables_initializer())

# Training cycle
for epoch in range(epochs):
    for i in range(0, n_batches):
        train_neural_network(sess, optimizer, volume_batches[i], labels_batches[i])
        print('Epoch {:>2}, Batch {}:  '.format(epoch + 1, i), end='')
        print_stats(sess, volume_batches[i], labels_batches[i], cost)